### 년도별 문장들 긍/부정 키워드 기반 카운팅


In [1]:
from counting_data.mariadb import MariaDB
with open('./counting_data/aws_config','r',encoding='utf-8') as f:
    db_config = {lines.split('=')[0].strip():lines.split('=')[1].strip() for lines in f.readlines()}
db = MariaDB(db_config)

In [6]:
import pandas as pd
import json

In [7]:
with open('./counting_data/total_kosdaq.json') as f:
    company_year_sentence = json.load(f)

In [8]:
# 키워드 설정
keywords = [('행복', 1), ('부정', 0), ('기쁘다',1),('자살',0),('긍정',1),('아니',0)]
pos_keywords = {kw for kw, val in keywords if val == 1}
neg_keywords = {kw for kw, val in keywords if val == 0}

# 
columns = ['company', 'year', 'pos_count', 'neg_count']
counting_df = pd.DataFrame(columns=columns)

In [9]:
index = 0
for company, years in company_year_sentence.items():
    for year, sentences in years.items():
        pos_count = 0
        neg_count = 0

        # 각 문장별 긍정 키워드 계산
        for sentence in sentences:
            if not isinstance(sentence, str):
                continue
            sentence_pos_count = 0
            for word in pos_keywords:
                sentence_pos_count += sentence.count(word)
            pos_count += sentence_pos_count

        # 각 문장별 부정 키워드 계산
        for sentence in sentences:
            if not isinstance(sentence, str):
                continue
            sentence_neg_count = 0
            for word in neg_keywords:
                sentence_neg_count += sentence.count(word)
            neg_count += sentence_neg_count

        counting_df.loc[index] = [company, year, pos_count, neg_count]
        index += 1

In [4]:
a = '안녕 나는'
b = '안녕'
a.count(b)
float 에러

1

In [23]:
company_count = (counting_df['company'],counting_df['year'],counting_df['pos_count'],counting_df['neg_count'])

,company,year,pos_count,neg_count
0,오상헬스케어,2019,0,1
1,오상헬스케어,2020,8,24
2,오상헬스케어,2021,2,16


In [22]:
db.insert_many('tb_company_keyword','Company,Year,Keyword,Pos_Count,Neg_Count',company_count)

True